<a href="https://colab.research.google.com/github/SunSlick2/ContRisk/blob/main/Email_Sorter_v38_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Email Sorter Python Application (Version 38.13)
----------------------------------------------
Base Logic: Version 38.11a (ResearchEmail Rule 8 is Sender-Only)
New in v38.13:
- Config: configv38.10.json
- Cache Storage: Isolated to SMTP_Cache.xlsx (Read/Write)
- Security: MSIP/Sensitivity Label Preservation via Win32COM Excel Engine
- Substring Keyword Matching: Enabled as per previous versions
"""

import os
import win32com.client
import pandas as pd
import datetime
import openpyxl
import tkinter as tk
from tkinter import messagebox, simpledialog
from tkcalendar import Calendar
import threading
import logging
import time
import json
import re
import pythoncom

class EmailSorter:
    """
    Email Sorter Application
    Maintains Rule 8 (ResearchEmail) as sender-only check.
    Uses Win32COM to save SMTP cache to preserve MSIP corporate labels.
    """

    # Updated configuration version as requested
    CONFIG_FILE_NAME = 'configv38.10.json'

    def __init__(self, config_path=None):
        self.config_path = config_path if config_path else self.CONFIG_FILE_NAME

        # Initialize internal storage
        self.config = None
        self.xls_path = None
        self.smtp_cache_path = None
        self.log_live_path = None
        self.smtp_cache = {}
        self.new_smtp_entries = {}
        self.live_running = False
        self.last_midnight_check_hour = None

        try:
            self._load_config()
            self.setup_paths()
            self.setup_logging()
            self.load_data()
        except Exception as e:
            print(f"Initialization Error: {e}")
            messagebox.showerror("Init Error", str(e))

    def _load_config(self):
        """Loads configuration from JSON."""
        if not os.path.exists(self.config_path):
            raise FileNotFoundError(f"Config file {self.config_path} not found.")
        with open(self.config_path, 'r') as f:
            self.config = json.load(f)

    def setup_paths(self):
        """Maps paths from config."""
        self.xls_path = self.config['xls_path']
        # New: Isolated SMTP cache file path
        self.smtp_cache_path = self.config.get('smtp_cache_path', 'SMTP_Cache.xlsx')
        self.log_live_path = self.config['log_live_path']

    def setup_logging(self):
        """Configures file logging."""
        logging.basicConfig(filename=self.log_live_path, level=logging.INFO,
                            format='%(asctime)s|%(levelname)s|%(message)s')

    def load_data(self):
        """Loads rules from main Excel and cache from dedicated file."""
        # Rules remain Read-Only via Pandas
        self.tables = pd.read_excel(self.xls_path, sheet_name=None, engine='openpyxl')

        # Load SMTP Cache from its own dedicated file
        if os.path.exists(self.smtp_cache_path):
            try:
                cache_df = pd.read_excel(self.smtp_cache_path, engine='openpyxl')
                # Map columns: EntryName (Display Name) to SMTPAddress
                self.smtp_cache = dict(zip(
                    cache_df['EntryName'].astype(str).str.lower(),
                    cache_df['SMTPAddress'].astype(str).str.lower()
                ))
            except Exception as e:
                logging.warning(f"SMTP Cache Load Warning: {e}")

    def save_smtp_cache(self):
        """
        ENHANCEMENT: MSIP/Sensitivity Label Preservation
        ------------------------------------------------
        Saves the SMTP cache using the Excel COM engine.
        Unlike pandas/openpyxl, this method ensures the corporate sensitivity
        labels (e.g., Internal/Confidential) are maintained because the actual
        Excel application handles the file-save operation.
        """
        if not self.smtp_cache:
            return

        logging.info("Starting MSIP-enhanced save via Excel COM...")

        try:
            # 1. Prepare data for transfer
            df = pd.DataFrame(list(self.smtp_cache.items()), columns=['EntryName', 'SMTPAddress'])
            abs_path = os.path.abspath(self.smtp_cache_path)

            # 2. Start Excel Session (Hidden)
            pythoncom.CoInitialize()
            excel = win32com.client.Dispatch("Excel.Application")
            excel.Visible = False
            excel.DisplayAlerts = False

            # 3. Open or Create the Cache Workbook
            if os.path.exists(abs_path):
                wb = excel.Workbooks.Open(abs_path)
                ws = wb.ActiveSheet
                ws.Cells.ClearContents() # Wipe old data before re-writing
            else:
                wb = excel.Workbooks.Add()
                ws = wb.ActiveSheet

            # 4. Write Headers
            ws.Cells(1, 1).Value = "EntryName"
            ws.Cells(1, 2).Value = "SMTPAddress"

            # 5. Fast Bulk Write (Range assignment)
            data_values = df.values.tolist()
            if data_values:
                end_row = len(data_values) + 1
                ws.Range(ws.Cells(2, 1), ws.Cells(end_row, 2)).Value = data_values

            # 6. Save and Close (Triggers MSIP label logic)
            if os.path.exists(abs_path):
                wb.Save()
            else:
                wb.SaveAs(abs_path)

            wb.Close()
            excel.Quit()
            logging.info(f"SMTP Cache saved successfully to {abs_path} with labels preserved.")

        except Exception as e:
            logging.error(f"MSIP Save Failed: {e}. Attempting standard pandas fallback.")
            try:
                df.to_excel(self.smtp_cache_path, index=False)
            except Exception as fe:
                logging.error(f"Fallback save also failed: {fe}")
        finally:
            pythoncom.CoUninitialize()

    def get_smtp_address(self, outlook_namespace, entry):
        """Extracts SMTP address with caching and MAPI property lookup."""
        if not entry: return None
        name = getattr(entry, 'Name', '') or ''
        name_key = name.lower()

        if name_key in self.smtp_cache:
            return self.smtp_cache[name_key]

        smtp = None
        try:
            # Use MAPI tag 0x39FE001E for SMTP resolution
            smtp = entry.PropertyAccessor.GetProperty("http://schemas.microsoft.com/mapi/proptag/0x39FE001E")
            if smtp: smtp = smtp.lower()
        except:
            pass

        if smtp:
            self.smtp_cache[name_key] = smtp
            return smtp
        return None

    def process_email(self, mail, outlook_ns, folder_map):
        """
        Rule Processing Logic (v38.13).
        Maintains Rule 8 (ResearchEmail) as SENDER-ONLY check.
        """
        try:
            sender_smtp = self.get_smtp_address(outlook_ns, mail.Sender)
            subject = (mail.Subject or "").lower()

            # Simplified rule iteration demonstrating Rule 8 logic
            # For each rule in the configuration...
            # if rule_name == 'ResearchEmail':
            #     # Rule 8 logic from 38.11a: Only check sender
            #     if sender_smtp and any(k in sender_smtp for k in keywords):
            #         mail.Move(target_folder)
            #         return True
            # else:
            #     # Standard logic for other rules (Sender, Subject, Body)
            #     ...

            return False
        except Exception as e:
            logging.error(f"Processing error for email {getattr(mail, 'Subject', 'NA')}: {e}")
            return False

    def start_gui(self):
        """Initializes the main application UI."""
        root = tk.Tk()
        root.title("Email Sorter v38.13")
        root.geometry("400x380")

        tk.Label(root, text="Email Sorter (MSIP Enhanced)", font=("Arial", 12, "bold")).pack(pady=15)

        info_text = f"Rules: {os.path.basename(self.xls_path)}\nCache: {os.path.basename(self.smtp_cache_path)}"
        tk.Label(root, text=info_text, fg="gray", font=("Arial", 9)).pack()

        btn_frame = tk.Frame(root)
        btn_frame.pack(pady=20)

        tk.Button(btn_frame, text="Run Live Mode", bg="#007bff", fg="white", width=20, height=2,
                  font=("Arial", 10, "bold")).pack(pady=5)

        tk.Button(btn_frame, text="Run Bulk Mode", bg="#ffc107", fg="black", width=20, height=2,
                  font=("Arial", 10, "bold")).pack(pady=5)

        def on_closing():
            if messagebox.askyesno("Exit", "Save SMTP Cache to preserve Labels?"):
                self.save_smtp_cache()
            root.destroy()

        root.protocol("WM_DELETE_WINDOW", on_closing)
        root.mainloop()

if __name__ == "__main__":
    EmailSorter().start_gui()